In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


# set mpl font sizing
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc("font", size=BIGGER_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
font_family = "serif"

# set seaborn styling
sns.set_style("whitegrid")
n_colors = 9
cms_color = {
    color: sns.color_palette(color, n_colors)
    for color in ["Blues", "Greens", "Reds", "Oranges", "Purples", "Greys"]
}

In [ ]:
from avapi.carla import CarlaScenesManager


data_dir = "/data/shared/CARLA/multi-agent-aerial"
CSM = CarlaScenesManager(data_dir=data_dir)
CDM = CSM.get_scene_dataset_by_index(0)

### Visualize Agent Locations

In [ ]:
from avstack.geometry import GlobalOrigin3D
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np


def plot_scene(agents, fovs, objs):
    # Define colormap
    cmap = plt.get_cmap(
        "tab10", len(agents)
    )  # Changed colormap for better color contrast

    # Create figure and axis
    fig, ax = plt.subplots(
        1, 1, figsize=(8, 8)
    )  # Increased figure size for better visibility

    # Loop over agents to plot each
    for i, agent in enumerate(agents):
        # Assign color based on colormap
        color = cmap(i)

        # Plot agent position with larger marker
        ax.scatter(
            x=agent.position.x[0],
            y=agent.position.x[1],
            label=f"Agent {i}",
            color=color,
            marker="o",
            s=100,  # Increased marker size for visibility
            edgecolor="black",  # Added border to the marker for clarity
        )

        # Add FOV rectangle with better transparency and border
        patch = Rectangle(
            xy=fovs[i]["xy"],
            width=fovs[i]["width"],
            height=fovs[i]["height"],
            angle=fovs[i]["angle"],
            rotation_point="center",
            facecolor=color,
            alpha=0.3,  # Adjusted transparency
            edgecolor="black",  # Added edge color for distinction
            linewidth=1,  # Set border width
        )
        ax.add_patch(patch)

    # Plot object positions with larger markers
    objs_pos = np.array([obj.position.x[:2] for obj in objs])
    ax.scatter(
        objs_pos[:, 0],
        objs_pos[:, 1],
        color="blue",
        marker="x",
        s=60,  # Increased marker size for objects
        linewidths=1.5,  # Increased line width of the marker for clarity
    )

    # Adding labels, grid, and legend
    # ax.set_xlabel("X")
    # ax.set_ylabel("Y")
    ax.set_title("BEV Scene View", fontweight="bold")
    ax.grid(
        visible=True, linestyle="--", alpha=0.5
    )  # Modified grid style for less prominence
    ax.legend(loc="upper right", fontsize=10)  # Added legend to distinguish agents

    # Display the plot
    plt.tight_layout()
    plt.show()


def load_agents_objects(idx_frame):
    # Get agent positions
    frame = CDM.get_frames(sensor="camera-0", agent=0)[idx_frame]
    agents = CDM.get_agents(frame)
    objs = CDM.get_objects_global(frame=frame, ignore_static_agents=True)

    # compute FOV given sensor parameters
    fovs = []
    for i in range(len(agents)):
        # get the half fov angles for each camera
        calib = CDM.get_calibration(frame=frame, sensor="camera-0", agent=i)
        ref_global = calib.reference.integrate(start_at=GlobalOrigin3D)
        d_az = np.arctan(calib.img_shape[1] / (2 * calib.P[0, 0]))
        d_el = np.arctan(calib.img_shape[0] / (2 * calib.P[1, 1]))
        dx = d_az * ref_global.x[2]
        dy = d_el * ref_global.x[2]

        # store the bottom left corner
        fov = {
            "xy": [-dx + agents[i].position.x[0], -dy + agents[i].position.x[1]],
            "angle": agents[i].attitude.yaw,
            "width": 2 * dx,
            "height": 2 * dy,
        }
        fovs.append(fov)

    return agents, fovs, objs

In [ ]:
for idx_frame in [0, 50, 100]:
    agents, fovs, objs = load_agents_objects(idx_frame=idx_frame)
    plot_scene(agents, fovs, objs)